# NGCC Turndown

This notebook runs a series of net electric power outputs from 650 MW to 160 MW (about 100% to 25%) for an NGCC with 97% CO2 capture. The NGCC model is based on the NETL report "Cost and Performance Baseline for Fossil Energy Plants Volume 1: Bituminous Coal and Natural Gas to Electricity." Sept 2019, Case B31B (https://www.netl.doe.gov/projects/files/CostAndPerformanceBaselineForFossilEnergyPlantsVol1BitumCoalAndNGtoElectBBRRev4-1_092419.pdf).

## Imports

Import the modules that will be used.

In [1]:
import os
import numpy as np
import pandas as pd
from IPython.core.display import SVG
import pyomo.environ as pyo
import idaes
from idaes.core.solvers import use_idaes_solver_configuration_defaults
import idaes.core.util.scaling as iscale
import idaes.core.util as iutil
import ngcc
import pytest

## Global Solver Settings

Using the IDAES configuration system for solver settings will apply these settings to all Ipopt solvers created, including the ones created in initialization methods.

In [2]:
use_idaes_solver_configuration_defaults()
idaes.cfg.ipopt.options.nlp_scaling_method = "user-scaling"
solver = pyo.SolverFactory("ipopt")

## Create the NGCC model

Create the NGCC model and initialize or read saved initialization.  The base initialized NGCC model is configured to mathc the baseline report.

In [ ]:
m = pyo.ConcreteModel()
m.fs = ngcc.NgccFlowsheet(default={"dynamic":False})
iscale.calculate_scaling_factors(m)
m.fs.initialize(
    load_from="data_init/ngcc_init.json.gz",
    save_to="data_init/ngcc_init.json.gz",
)
res = solver.solve(m, tee=True)

2022-03-10 21:08:07 [INFO] idaes.init.fs: NGCC load initial from data_init/ngcc_init.json.gz


## Show PFDs with baseline results

This displays PFDs in the notebook, and saves them to files.  The full NGCC model is too big to show in a single PFD, so they are broken into the three main sections, gas turbine, heat recovery steam generator (HRSG), and steam turbine.

In [ ]:
display(SVG(m.fs.gt.write_pfd()))
display(SVG(m.fs.hrsg.write_pfd()))
display(SVG(m.fs.st.write_pfd()))

m.fs.gt.write_pfd(fname="data_pfds/gt_baseline.svg")
m.fs.hrsg.write_pfd(fname="data_pfds/hrsg_baseline.svg")
m.fs.st.write_pfd(fname="data_pfds/st_baseline.svg")

## Show some key model outputs

Show some key outputs to make sure the baseline model argrees with the baseline report.

In [ ]:
m.fs.gross_power.display()
m.fs.gt.gt_power.display()
m.fs.st.steam_turbine.power.display()
m.fs.lhv_efficiency.display()

#Assert results approximatly agree with baseline reoprt
assert pyo.value(m.fs.gross_power[0]) == pytest.approx(-690e6, rel=0.001)
assert pyo.value(100*m.fs.lhv_efficiency[0]) == pytest.approx(52.8, abs=0.1)
assert pyo.value(m.fs.gt.gt_power[0]) == pytest.approx(-477e6, rel=0.001)

## Run turndown cases 5 MW interval

Here we set the CO2 capture rate to 97% and set the specific reboiler duty to PZ advanced solvent system. 160 MW net corresponds to a bit under 25% which is approximatly the minimum turndown for an F-Class NGCC.  This also tabulates the results in the NGCC tags_output tag group. 

In [ ]:
idaes.cfg.ipopt.options.tol = 1e-6
idaes.cfg.ipopt.options.max_iter = 50
idaes.cfg.ipopt.options.ma27_pivtol = 1e-2
idaes.cfg.ipopt.options.halt_on_ampl_error = "no"
solver = pyo.SolverFactory("ipopt")

m.fs.cap_specific_reboiler_duty.fix(2.4e6)
m.fs.cap_fraction.fix(0.97)
powers = np.linspace(650, 160, int((650 - 160)/5) + 1)

df = pd.DataFrame(columns=m.fs.tags_output.table_heading())

for p in powers:
    print(p)
    fname = f"data/ngcc_{int(p)}.json.gz"
    if os.path.exists(fname):
        iutil.from_json(m, fname=fname, wts=iutil.StoreSpec(suffix=False))
    else:
        m.fs.net_power_mw.fix(p)
        res = solver.solve(m, tee=True, symbolic_solver_labels=True)
        if not pyo.check_optimal_termination(res):
            break
        iutil.to_json(m, fname=fname)
    df.loc[m.fs.tags_output["net_power"].value] = m.fs.tags_output.table_row(numeric=True)

## Save turndown results

Save the results to a CSV file and view the head and tail of the results data frame. 

In [ ]:
df.to_csv("data_tabulated/ngcc.csv")
df